# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.event_detection import detect_rsw_and_rcap

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import get_stops_utils, collect_stop_data

from neural_data_analysis.neural_analysis_tools.glm_tools.glm_fit import general_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained, glm_runner
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func, compare_glm_fit
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import assemble_stop_design

# import decoding
from neural_data_analysis.neural_analysis_tools.decoding_tools.event_decoding import decoding_utils, decoding_analysis, plot_decoding, cmp_decode, load_results
from neural_data_analysis.design_kits.design_by_segment import spike_history

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_colwidth', 200)

# retrieve data

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0301"

pn, datasets, comparisons = collect_stop_data.collect_stop_data_func(
    raw_data_folder_path)

globals().update(datasets)

captures_df, valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    distance_col="distance_from_ff_to_stop",
)

# Get stop info

## new_seg_info

In [ ]:
stops_with_stats['stop_time'] = stops_with_stats['stop_id_start_time']
stops_with_stats['prev_time'] = stops_with_stats['stop_id_end_time'].shift(1)
stops_with_stats['next_time'] = stops_with_stats['stop_id_start_time'].shift(-1)

new_seg_info = event_binning.make_new_seg_info_for_stop_design(stops_with_stats, pn.closest_stop_to_capture_df, pn.monkey_information)


events_with_stats = stops_with_stats[['stop_id','stop_cluster_id','stop_id_start_time','stop_id_end_time']].copy()
events_with_stats.rename(columns={'stop_id':'event_id', 'stop_cluster_id':'event_cluster_id', 
                                  'stop_id_start_time':'event_id_start_time', 
                                  'stop_id_end_time':'event_id_end_time'}, inplace=True)

## what if only keep stop clusters w captured

In [ ]:
# cluster_id_w_captures = new_seg_info.loc[new_seg_info['captured'] == 1, 'stop_cluster_id'].unique()
# new_seg_info = new_seg_info[new_seg_info['stop_cluster_id'].isin(cluster_id_w_captures)].copy()

In [ ]:
# new_seg_info = new_seg_info[new_seg_info['captured'] == 1].copy()
# new_seg_info

# Prepare for GLM

## binned_feats

In [ ]:
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import assemble_stop_design

In [ ]:

binned_spikes_og, binned_feats_og, offset_log_og, meta_used_og, meta_groups = assemble_stop_design.build_stop_design(new_seg_info, events_with_stats, 
                                                                             pn.monkey_information, 
                                                                             pn.spikes_df, pn.ff_dataframe, 
                                                                             datasets=datasets,
                                                                             bin_dt=pn.bin_width, add_ff_visible_info=True)

binned_feats_og['random_bool'] = np.random.rand(len(binned_feats_og)) > 0.5 # do this, sort of as a control

In [ ]:
binned_feats = binned_feats_og.copy()
binned_spikes = binned_spikes_og.copy()
offset_log = offset_log_og
meta_used = meta_used_og.copy()

### take out subsets if needed

In [ ]:
# # Take out stops that are misses
# binned_feats, binned_spikes, offset_log, meta_used = assemble_stop_design.subset_binned_data(
#     binned_feats_og, binned_spikes_og, offset_log_og, meta_used_og, 
#     np.asarray(binned_feats_og['miss']==1)
# )

# binned_feats.drop(columns=['miss'], inplace=True)

# # also need to drop columns that involve 'captured'
# cols_to_drop = [c for c in binned_feats.columns if 'captured' in c]
# binned_feats = binned_feats.drop(columns=cols_to_drop)

# # also drop 'rcap_last' if present
# if 'rcap_last' in binned_feats.columns:
#     binned_feats = binned_feats.drop(columns=['rcap_last'])
    
# # actually, just drop all columns that involve rsw or rcap
# cols_to_drop = [c for c in binned_feats.columns if 'rsw' in c or 'rcap' in c or 'miss' in c]
# binned_feats = binned_feats.drop(columns=cols_to_drop)

In [ ]:
# # Take out stops that are not misses
# binned_feats, binned_spikes, offset_log, meta_used = assemble_stop_design.subset_binned_data(
#     binned_feats_og, binned_spikes_og, offset_log_og, meta_used_og, 
#     np.asarray(~binned_feats_og['miss']==1)
# )

# cols_to_drop = ['miss', 'rsw_first', 'rcap_first', 'rsw_middle', 'rcap_middle', 'rsw_last', 'one_stop_miss']
# binned_feats.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# # Take out stops that are not retries
# binned_feats, binned_spikes, offset_log, meta_used = assemble_stop_design.subset_binned_data(
#     binned_feats_og, binned_spikes_og, offset_log_og, meta_used_og, 
#     np.asarray(binned_feats_og['whether_in_retry_series']==0)
# )

# cols_to_drop = ['whether_in_retry_series', 'next_gap_s_z', 'rsw_first', 'rcap_first', 'rsw_middle', 'rcap_middle', 'rsw_last', 'one_stop_miss']
# binned_feats.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# # take out second half of the data
# binned_feats = binned_feats_og.loc[15000:].copy()
# binned_spikes = binned_spikes_og.loc[15000:].copy()
# offset_log = offset_log_og[15000:].copy()
# meta_used = meta_used_og.loc[15000:].copy()


In [ ]:
# # Take out random stops
# binned_feats, binned_spikes, offset_log, meta_used = assemble_stop_design.subset_binned_data(
#     binned_feats_og, binned_spikes_og, offset_log_og, meta_used_og, 
#     np.asarray(binned_feats_og['random_bool'])
# )

# binned_feats.drop(columns=['random_bool'], inplace=True)

In [ ]:
binned_feats_og.shape

In [ ]:
binned_feats.shape

### add interaction

In [ ]:
binned_feats = assemble_stop_design.add_interaction_columns(binned_feats)

### scale

In [ ]:
binned_feats_sc = assemble_stop_design.scale_binned_feats(binned_feats)

In [ ]:
df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

## see features

## check VIF

In [ ]:
binned_feats_sc = binned_feats_sc.drop(columns=['next_gap_s_z_x_retry', 'cluster_rel_time_s_z_x_retry', 
                                                'cluster_progress_c_x_retry', 'event_is_first_in_cluster_x_retry'], 
                                       errors='ignore')

In [ ]:
from neural_data_analysis.topic_based_neural_analysis.full_session import selected_raw_data_features, selected_pn_design_features, selected_stop_design_features
print(f'set(binned_feats_sc.columns) - set(selected_stop_design_features.full_stop_design_predictors): {set(binned_feats_sc.columns) - set(selected_stop_design_features.full_stop_design_predictors)}')
binned_feats_sc_sub = binned_feats_sc[selected_stop_design_features.full_stop_design_predictors]

In [ ]:
reload(design_checks)

In [ ]:
binned_feats_sc_sub.shape

In [ ]:
cols_path = os.path.join(pn.planning_and_neural_folder_path, 'glm_around_stops', 'selected_cols.json')
X_pruned, vif_report = design_checks.load_or_compute_selected_cols(binned_feats_sc_sub, cols_path, exists_ok=True)

## drop columns that involve 'captured'

In [ ]:
# cols_to_drop = [c for c in binned_feats_sc.columns if 'captured' in c]
# binned_feats_sc = binned_feats_sc.drop(columns=cols_to_drop)
# binned_feats_sc.describe()
# ## check df

# Get spike history

In [ ]:
bin_df = spike_history.make_bin_df_from_stop_meta(meta_used)
dt = pn.bin_width
t_max = 0.20

design_w_history, basis, colnames, meta_groups = (
    spike_history.build_design_with_spike_history_from_bins(
        spikes_df=pn.spikes_df,
        bin_df=bin_df,
        X_pruned=X_pruned,
        meta_groups=meta_groups,
        dt=dt,
        t_max=t_max,
    )
)


In [ ]:
# new_seg_info['new_segment'] = new_seg_info['event_id']

# GLM

## class

In [ ]:
meta_used[['bin', 't_left', 't_right']]

In [ ]:
df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

output_root = os.path.join(pn.planning_and_neural_folder_path, 'glm_around_stops')

pipeline = glm_runner.GLMPipeline(
    spikes_df=pn.spikes_df,
    bin_df=bin_df,
    df_X=df_X,
    df_Y=df_Y,
    meta_groups=meta_groups,
    bin_width=pn.bin_width,
    output_root=output_root,
)

pipeline.run()
pipeline.plot_comparisons()


In [ ]:
stop!

## just behavioral vars

In [ ]:
df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

In [ ]:
save_dir = os.path.join(pn.planning_and_neural_folder_path, 'glm_around_stops', 'glm_fit')

report0 = general_glm_fit.glm_mini_report(
    df_X=X_pruned, df_Y=df_Y, offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=True, 
    make_plots=True,
    show_plots=True,
    save_dir=save_dir,
)

In [ ]:
coefs_df = report0['coefs_df']
coefs_df[(coefs_df['term'] == 'captured') & (coefs_df['sig_FDR'] == True)].sort_values('p', ascending=True)

In [ ]:
coefs_df['refit_on_support'].value_counts()

In [ ]:
coefs_df.info()

## both (behav and spike history)

In [ ]:
cols_path = os.path.join(pn.planning_and_neural_folder_path, 'glm_around_stops', 'selected_cols_w_spike_history.json')
try:
    with open(cols_path, 'r') as f:
        selected_cols_w_history = json.load(f)
    X_pruned1 = design_w_history[selected_cols_w_history].copy()
    print(f'Loaded selected columns from {cols_path}')
except:
    os.makedirs(os.path.dirname(cols_path), exist_ok=True)
    X_pruned1, vif_report = design_checks.check_design(design_w_history)
    with open(cols_path, 'w') as f:
        json.dump(X_pruned1.columns.tolist(), f)
    print(f'Saved selected columns to {cols_path}')

In [ ]:
save_dir = os.path.join(pn.planning_and_neural_folder_path, 'glm_around_stops', 'glm_fit')

report1 = general_glm_fit.glm_mini_report(
    df_X=X_pruned1, df_Y=df_Y, 
    offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=True, 
    make_plots=True,
    show_plots=True,
    meta_groups=meta_groups,
    save_dir=save_dir,
)

## just spike history

In [ ]:
all_history_cols = [c for c in design_w_history.columns if (c.startswith('cluster_') 
                                                            and c not in binned_feats_sc.columns)]
cols_path = os.path.join(pn.planning_and_neural_folder_path, 'glm_around_stops', 'selected_spike_history_cols.json')
try:
    with open(cols_path, 'r') as f:
        selected_history_cols = json.load(f)
    X_pruned2 = design_w_history[selected_history_cols].copy()
    print(f'Loaded selected columns from {cols_path}')
except:
    os.makedirs(os.path.dirname(cols_path), exist_ok=True)
    X_pruned2, vif_report = design_checks.check_design(design_w_history[all_history_cols])
    with open(cols_path, 'w') as f:
        json.dump(X_pruned2.columns.tolist(), f)
    print(f'Saved selected columns to {cols_path}')

In [ ]:
save_dir = os.path.join(pn.planning_and_neural_folder_path, 'glm_around_stops', 'glm_fit')

report2 = general_glm_fit.glm_mini_report(
    df_X=X_pruned2, df_Y=df_Y, 
    offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=True, 
    make_plots=True,
    show_plots=True,
    meta_groups=meta_groups,
    save_dir=save_dir,
)

# Compare deviance explained

## In-sample

In [ ]:
metrics_by_model = {
    'Behavior only': report0['metrics_df'],
    'Behavior + history': report1['metrics_df'],
    'History only': report2['metrics_df'],
}

compare_glm_fit.plot_insample_model_comparison(metrics_by_model)


## CV

In [ ]:
metrics_by_model = {
    'Behavior only': report0['metrics_df'],
    'Behavior + history': report1['metrics_df'],
    'History only': report2['metrics_df'],
}

compare_glm_fit.plot_cv_model_comparison(metrics_by_model)


# Deviance explained

## in sample

In [ ]:
plot_glm_fit.plot_insample_model_diagnostics(
    report0['metrics_df'],
)

In [ ]:
plot_glm_fit.plot_insample_model_diagnostics(
    report1['metrics_df'],
)

In [ ]:
plot_glm_fit.plot_insample_model_diagnostics(
    report2['metrics_df'],
)

In [ ]:
plot_glm_fit.plot_cv_model_diagnostics(
    report0['metrics_df'],
    bins=20,
    show=True,
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

metrics_df = report['metrics_df'].copy()

# ---- derived quantities ----
metrics_df['ll_improvement'] = metrics_df['llf'] - metrics_df['llnull']
metrics_df['ll_improvement_per_obs'] = metrics_df['ll_improvement'] / metrics_df['n_obs']

# ---- figure layout ----
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.ravel()

# ========== 1. Deviance explained distribution ==========
axes[0].hist(metrics_df['deviance_explained'], bins=20)
axes[0].axvline(metrics_df['deviance_explained'].median(), linestyle='--')
axes[0].set_xlabel('Deviance explained')
axes[0].set_ylabel('Number of neurons')
axes[0].set_title('Model performance (deviance explained)')

# ========== 2. McFadden R² distribution ==========
axes[1].hist(metrics_df['mcfadden_R2'], bins=20)
axes[1].axvline(metrics_df['mcfadden_R2'].median(), linestyle='--')
axes[1].set_xlabel('McFadden $R^2$')
axes[1].set_ylabel('Number of neurons')
axes[1].set_title('Pseudo-$R^2$ distribution')

# ========== 3. Deviance explained vs McFadden R² ==========
axes[2].scatter(
    metrics_df['deviance_explained'],
    metrics_df['mcfadden_R2'],
    alpha=0.7
)
axes[2].set_xlabel('Deviance explained')
axes[2].set_ylabel('McFadden $R^2$')
axes[2].set_title('Consistency check')

# ========== 4. Deviance explained vs null deviance ==========
axes[3].scatter(
    metrics_df['null_deviance'],
    metrics_df['deviance_explained'],
    alpha=0.7
)
axes[3].set_xlabel('Null deviance (rate / variability proxy)')
axes[3].set_ylabel('Deviance explained')
axes[3].set_title('Dependence on firing statistics')

# ========== 5. Log-likelihood improvement ==========
axes[4].hist(metrics_df['ll_improvement'], bins=20)
axes[4].axvline(0, linestyle='--')
axes[4].set_xlabel('Log-likelihood improvement')
axes[4].set_ylabel('Number of neurons')
axes[4].set_title('Improvement over null model')

# ========== 6. LL improvement per observation ==========
axes[5].hist(metrics_df['ll_improvement_per_obs'], bins=20)
axes[5].axvline(0, linestyle='--')
axes[5].set_xlabel('Δ log-likelihood per observation')
axes[5].set_ylabel('Number of neurons')
axes[5].set_title('Predictive gain (normalized)')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

metrics_df = report['metrics_df'].copy()

# ---- derived quantities ----
metrics_df['ll_improvement'] = metrics_df['llf'] - metrics_df['llnull']
metrics_df['ll_improvement_per_obs'] = metrics_df['ll_improvement'] / metrics_df['n_obs']

# ---- figure layout ----
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.ravel()

# ========== 1. Deviance explained distribution ==========
axes[0].hist(metrics_df['deviance_explained'], bins=20)
axes[0].axvline(metrics_df['deviance_explained'].median(), linestyle='--')
axes[0].set_xlabel('Deviance explained')
axes[0].set_ylabel('Number of neurons')
axes[0].set_title('Model performance (deviance explained)')

# ========== 2. McFadden R² distribution ==========
axes[1].hist(metrics_df['mcfadden_R2'], bins=20)
axes[1].axvline(metrics_df['mcfadden_R2'].median(), linestyle='--')
axes[1].set_xlabel('McFadden $R^2$')
axes[1].set_ylabel('Number of neurons')
axes[1].set_title('Pseudo-$R^2$ distribution')

# ========== 3. Deviance explained vs McFadden R² ==========
axes[2].scatter(
    metrics_df['deviance_explained'],
    metrics_df['mcfadden_R2'],
    alpha=0.7
)
axes[2].set_xlabel('Deviance explained')
axes[2].set_ylabel('McFadden $R^2$')
axes[2].set_title('Consistency check')

# ========== 4. Deviance explained vs null deviance ==========
axes[3].scatter(
    metrics_df['null_deviance'],
    metrics_df['deviance_explained'],
    alpha=0.7
)
axes[3].set_xlabel('Null deviance (rate / variability proxy)')
axes[3].set_ylabel('Deviance explained')
axes[3].set_title('Dependence on firing statistics')

# ========== 5. Log-likelihood improvement ==========
axes[4].hist(metrics_df['ll_improvement'], bins=20)
axes[4].axvline(0, linestyle='--')
axes[4].set_xlabel('Log-likelihood improvement')
axes[4].set_ylabel('Number of neurons')
axes[4].set_title('Improvement over null model')

# ========== 6. LL improvement per observation ==========
axes[5].hist(metrics_df['ll_improvement_per_obs'], bins=20)
axes[5].axvline(0, linestyle='--')
axes[5].set_xlabel('Δ log-likelihood per observation')
axes[5].set_ylabel('Number of neurons')
axes[5].set_title('Predictive gain (normalized)')

plt.tight_layout()
plt.show()


# More analysis of glm results

In [ ]:
report.keys()

In [ ]:
report['metrics_df']

## Note large coeff

In [ ]:
glm_fit_utils.summarize_large_coeffs(report['coefs_df'], df_X)

## Get predictions

In [ ]:
reload(variance_explained)
df_Y_pred = variance_explained.build_df_Y_pred_from_results(
    results=report['results'],
    df_X=df_X,
    offset_log=offset_log,
    df_Y=df_Y
)

# sanity checks
assert df_Y_pred.shape == df_Y.shape
assert np.isfinite(df_Y_pred.values).all()

In [ ]:
# Convert observed/predicted DataFrames to arrays
X = df_Y.to_numpy()      # observed counts
X_hat = df_Y_pred.to_numpy() # predicted expected counts
event_ids = meta_used['event_id'].to_numpy()


## population latent

In [ ]:
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_eval import shared_manifold, parity_utils, population_latent_benchmark

In [ ]:
# time_idx = data['new_bin']
time_idx = meta_used['rel_center'].values
condition_labels = None
trial_ids = meta_used['event_id'].values

res = population_latent_benchmark.population_latent_benchmark_function(
    X, X_hat, trial_ids, time_idx, condition_labels,
    fa_factors=8,
    max_rank=10,
    shuffle_mode='latent'
)

res['figs']['cca_spectrum']
res['figs']['ve_summary']
res['figs']['rrr_curve']


## shared components

Does the GLM recover the same latent population dynamics that are present in the real neural data?

Are the dominant latent trajectories aligned?

Are they time-locked similarly?

Are the shared dimensions smooth, structured, and consistent across trials?

Your plots are asking:

If I look at neural activity through the lens of shared canonical components, do real data and model predictions trace the same paths?”

In [ ]:

from neural_data_analysis.neural_analysis_tools.glm_tools.glm_eval.shared_manifold import plot_shared_components
import pandas as pd

df_lat = pd.concat(
    res['parity']['latents_by_fold'].values(),
    ignore_index=True
)

best_rank = res['rrr']['best_rank']

figs = plot_shared_components(
    df_lat,
    components=[1, 2], # or use n_components=best_rank instead
    overlay_mode='separate'
)


## shared phase planes

In [ ]:
# Option 1: one fold
fig = shared_manifold.plot_shared_phase_plane(
    res['parity']['latents_by_fold'][0],
    components=(1, 2),
    smooth_bins=3
)

# or concat all folds (one caveat: Different folds use slightly different CCA fits.)

df_lat = pd.concat(
    res['parity']['latents_by_fold'].values(),
    ignore_index=True
)

fig = shared_manifold.plot_shared_phase_plane(
    df_lat,
    components=(1, 2),
    smooth_bins=3
)


fig = shared_manifold.plot_shared_phase_plane_by_fold(
    res['parity']['latents_by_fold'],
    components=(1, 2),
    smooth_bins=3,
    alpha_fold=0.6
)


## Plot spikes

In [ ]:
reload(plot_spikes)

cluster_idx = 6

for event_id in range(30, 33):
    if event_id in meta_used['event_id'].values:
        # If your GLM used offset_log = np.log(exposure_s), you can omit exposure_s:
        plot_spikes.plot_observed_vs_predicted_event(
            binned_feats_sc=binned_feats_sc,
            binned_spikes=binned_spikes,
            meta_used=meta_used,
            offset_log=offset_log,
            model_res=report['results'][cluster_idx],   # GLM for cluster 0
            cluster_idx=cluster_idx,
            seg_id=event_id,
            seg_col='event_id',
            time_col='rel_center'
        )

## tuning curve

In [ ]:
report['coefs_df']['term'].unique()

In [ ]:
reload(plot_tuning_func)

cluster_id = 10
model_res = report['results'][cluster_id]
var = 'captured'

exposure_s = np.exp(offset_log)
# make empirical curve
tc_emp = plot_tuning_func.empirical_tuning_curve(
    binned_spikes=binned_spikes[cluster_id].to_numpy(),
    predictor_vals=binned_feats[var].to_numpy(),
    exposure_s=exposure_s,
    nbins=20
)

# make GLM curve
tc_glm = plot_tuning_func.glm_tuning_curve(
    model_res, df_X,
    var=var,
    offset_log=offset_log,
    average='marginal',
    weights=exposure_s,                 # time-weighted average rate (recommended)
    return_ci=True
)

# overlay
plot_tuning_func.overlay_tuning_curves(tc_emp, tc_glm, xcol=var,
                      title=f'Unit {cluster_id}: {var} tuning')

In [ ]:
plot_tuning_func.plot_tuning_with_ci(tc_glm, xcol='captured', ycol='rate_hz')
plt.show()

## plot pred_vs_obs

In [ ]:
cluster_id = 5
cv_stop_glm.plot_pred_vs_obs(report['results'][cluster_id], df_X, binned_spikes[cluster_id], offset_log)

# CCA

In [ ]:
cca_inst = cca_class.CCAclass(X1=binned_spikes.copy(), X2=binned_feats_sc.copy(), lagging_included=False)

In [ ]:
cca_inst.conduct_cca()

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X2', squared=False)

# check fr

In [ ]:
dt = 0.04
rates = binned_spikes.sum(axis=0) / (len(binned_spikes) * dt)
# rates is a Series indexed by unit, in Hz
rates

In [ ]:
import pandas as pd

def firing_rates_from_df(spikes_df, time_col='time', cluster_col='cluster'):
    start_s = spikes_df[time_col].min()
    end_s = spikes_df[time_col].max()
    duration = end_s - start_s

    counts = spikes_df.groupby(cluster_col).size()
    rates_hz = counts / duration
    return rates_hz.rename('rate_hz').reset_index()

firing_rates_from_df(pn.spikes_df)

# Inspect data

In [ ]:
new_seg_info

In [ ]:
seg_sub = new_seg_info[new_seg_info['captured'] > 0]
sns.histplot(new_seg_info['n_pre_bins'])
sns.histplot(seg_sub['n_pre_bins'])

In [ ]:
sns.histplot(new_seg_info['n_post_bins'])
sns.histplot(seg_sub['n_post_bins'])

# plot_spaghetti_per_stop

## run func

In [ ]:
cols = ['event_id', 'rel_center', 't_left', 't_right']
binned_spikes2 = binned_spikes.copy()
binned_spikes2[cols] = meta_used[cols]

In [ ]:
binned_spikes

In [ ]:
# choose a unit column by name or int (e.g., 3)
# unit_col = 3  # or '3' if your columns are strings
for unit_col in binned_spikes.columns:
    df_rate = plot_spikes.make_rate_df_from_binned(binned_spikes2, unit_col)

    # plot (with gentle smoothing and pre-stop baseline subtraction)
    fig, ax, n = plot_spikes.plot_spaghetti_per_stop(
        df_rate,
        smooth_sigma_s=0.08,          # ~80 ms sigma (auto-converted to bins)
        # baseline_window=(-0.5, -0.1), # subtract mean pre-stop activity
        baseline_window=None,
        max_stops=None,               # or an int to limit how many lines
        median_label='median (all stops)',
        title=f'Unit {unit_col}: rate per stop'
    )
    plt.show()
    print(f'Plotted {n} stops.')
    
    break
    

# Hyperparam tuning (try different configs)

## just elastic net

In [ ]:
cluster_df

In [ ]:
reload(general_glm_fit)

In [ ]:


res = general_glm_fit.glm_mini_report(
    df_X, df_Y, offset_log,
    regularization='elasticnet',
    # alpha_grid=(0.05, 0.1, 0.2, 0.5, 1.0),
    # l1_wt_grid=(0.75, 0.5, 0.25, 0.0),
    alpha_grid=(0.1, 0.2),
    l1_wt_grid=(0.75, 0.5),
    groups=cluster_df['event_cluster_id'].values
)

In [ ]:
res.keys()

In [ ]:
res['cv_tables_df']

In [ ]:
res['metrics_df']

In [ ]:
res['figures']['rr_hist']

In [ ]:
# all chosen alphas
print(res['metrics_df']['alpha'].value_counts())

# mean/median chosen alpha across clusters
print(res['metrics_df']['alpha'].agg(['mean','median']))

# joint distribution of alpha × l1_wt
pd.crosstab(res['metrics_df']['alpha'], res['metrics_df']['l1_wt'])

## configs (systematic)

In [ ]:
configs = [
    # 1) Ridge (ultra-light grid) ─ combos: ~18
    dict(name='Poisson_Ridge_ultra',
         regularization='elasticnet',
         alpha_grid=tuple(np.concatenate([
             np.logspace(-6, -4.7, 6),   # 1e-6 … ~2e-5
             np.logspace(-4.7, -2.3, 12) # ~2e-5 … 5e-3
         ])),
         l1_wt_grid=(0.0,),                 # pure L2
         cv_metric='loglik', n_splits=5, cov_type='HC1',
         refit_on_support=False),

    # 2) Lasso (ultra-light grid) + refit ─ combos: ~16
    dict(name='Poisson_Lasso_ultra_refit',
         regularization='elasticnet',
         alpha_grid=tuple(np.concatenate([
             np.logspace(-6, -4.7, 6),   # 1e-6 … ~2e-5
             np.logspace(-4.7, -3.3, 10) # ~2e-5 … 5e-4
         ])),
         l1_wt_grid=(1.0,),                 # pure L1
         cv_metric='loglik', n_splits=5, cov_type='HC1',
         refit_on_support=True),
    
    
    # 0) Plain Poisson MLE, robust SEs (baseline) ─ combos: 1
    dict(name='Poisson_MLE_HC3',
         regularization='none',
         alpha_grid=(0.0,), l1_wt_grid=(0.0,),
         cv_metric='loglik', n_splits=5, cov_type='HC3',
         refit_on_support=False),

]

In [ ]:
# configs = [
#     # 0) Plain Poisson MLE, robust SEs (baseline)  ─ combos: 1
#     dict(name='Poisson_MLE_HC3',
#          regularization='none',
#          alpha_grid=(0.0,), l1_wt_grid=(0.0,),
#          cv_metric='loglik', n_splits=5, cov_type='HC3',
#          refit_on_support=False),

#     # 1) Ridge (fine grid) ─ combos: 20
#     dict(name='Poisson_Ridge_fine',
#          regularization='elasticnet',
#          alpha_grid=tuple(np.logspace(-4, 2, 20)),
#          l1_wt_grid=(0.0,),                 # pure L2
#          cv_metric='loglik', n_splits=5, cov_type='HC1',
#          refit_on_support=False),

#     # 2) Lasso (fine) + refit ─ combos: 20
#     dict(name='Poisson_Lasso_refit_fine',
#          regularization='elasticnet',
#          alpha_grid=tuple(np.logspace(-4, 0, 20)),
#          l1_wt_grid=(1.0,),                 # pure L1
#          cv_metric='loglik', n_splits=5, cov_type='HC1',
#          refit_on_support=True),

#     # 3) Elastic-Net (mix) ─ combos: 14 × 5 = 70  ← HEAVY
#     dict(name='Poisson_EN_fine',
#          regularization='elasticnet',
#          alpha_grid=tuple(np.logspace(-4, 1, 14)),
#          l1_wt_grid=(1.0, 0.75, 0.5, 0.25, 0.0),
#          cv_metric='loglik', n_splits=5, cov_type='HC1',
#          refit_on_support=True),

#     # 4) EN, time-aware CV ─ combos: 12 × 3 = 36
#     dict(name='Poisson_EN_timecv',
#          regularization='elasticnet',
#          alpha_grid=tuple(np.logspace(-4, 0, 12)),
#          l1_wt_grid=(1.0, 0.5, 0.0),
#          cv_metric='loglik', n_splits=5, cov_type='HC1',
#          refit_on_support=True,
#          cv_splitter='blocked_time'),

#     # 5) Ridge, time-aware CV ─ combos: 16
#     dict(name='Poisson_Ridge_timecv',
#          regularization='elasticnet',
#          alpha_grid=tuple(np.logspace(-4, 1, 16)),
#          l1_wt_grid=(0.0,),
#          cv_metric='loglik', n_splits=5, cov_type='HC1',
#          refit_on_support=False,
#          cv_splitter='blocked_time'),

#     # 6) Lasso (HC3) + refit ─ combos: 16
#     dict(name='Poisson_Lasso_HC3_refit',
#          regularization='elasticnet',
#          alpha_grid=tuple(np.logspace(-4, 0, 16)),
#          l1_wt_grid=(1.0,),
#          cv_metric='loglik', n_splits=5, cov_type='HC3',
#          refit_on_support=True),

#     # 7) EN (deviance CV metric) ─ combos: 10 × 4 = 40
#     dict(name='Poisson_EN_devianceCV',
#          regularization='elasticnet',
#          alpha_grid=tuple(np.logspace(-3, 0.7, 10)),
#          l1_wt_grid=(1.0, 0.5, 0.25, 0.0),
#          cv_metric='deviance', n_splits=5, cov_type='HC1',
#          refit_on_support=True),

#     # 8) Quasi-Poisson flavor (same mean; inflated SEs) ─ combos: 1
#     dict(name='QuasiPoisson_like',
#          regularization='none',
#          alpha_grid=(0.0,), l1_wt_grid=(0.0,),
#          cv_metric='deviance', n_splits=5, cov_type='HC3',
#          refit_on_support=False,
#          use_overdispersion_scale=True),
# ]

## compare (w class)

In [ ]:
# 1) Build your configs (as before)
configs = [
    dict(name='Poisson_MLE_HC3',
         regularization='none',
         alpha_grid=(0.0,), l1_wt_grid=(0.0,),
         cv_metric='loglik', n_splits=5, cov_type='HC3',
         refit_on_support=False),
    dict(name='Poisson_Ridge_fine',
         regularization='elasticnet',
         alpha_grid=tuple(np.logspace(-4, 2, 20)),
         l1_wt_grid=(0.0,),
         cv_metric='loglik', n_splits=5, cov_type='HC1',
         refit_on_support=False),
    # ...
]

# 2) Spin up the runner
runner = glm_hyperparams_class.SweepAndCompare(
    configs=configs,
    df_X=df_X,
    df_Y=df_Y,
    offset_log=offset_log,
    fit_fn=glm_hyperparams_class.fit_fn,                           # <-- the adapter above
    feature_names=list(df_X.columns),
    cluster_ids=list(df_Y.columns),
    groups=cluster_df['event_cluster_id'].values,
    cov_type='HC1',
    out_dir='glm_sweep_compare_out',
    autosave=True,
    autosave_every=1,
    extra_fit_kwargs=dict(
        cv_splitter=None,                    # or 'blocked_time'
        use_overdispersion_scale=False,
        add_outer_cv_summary=True,           # optional: outer-CV summary
    ),
)

# 3) Run (and safely interrupt/resume any time)
runner.run()
summary_df = runner.get_summary()
cv_tables_df   = runner.get_cv_tables()
results = runner.per_config_results

In [ ]:
runner.per_config_results  # list of dicts (detailed)

# resume later from checkpoint:
resumed = glm_hyperparams_class.SweepAndCompare.load_from_checkpoint(
    checkpoint_dir=runner._ckpt_dir,
    df_X=df_X, df_Y=df_Y, offset_log=offset_log, fit_fn=glm_hyperparams_class.fit_fn,
    feature_names=list(df_X.columns), cluster_ids=list(df_Y.columns), groups=cluster_df['event_cluster_id'].values
)
resumed.run()   # continues only the unfinished configs

## win

In [ ]:
# cv_tables_df = pd.read_csv("all_monkey_data/glm_runs/glm_sweep_2025-09-06/cv_tables.csv")
# summary_df = pd.read_csv("all_monkey_data/glm_runs/glm_sweep_2025-09-06/summary.csv")

In [ ]:
cv_all = cv_tables_df  # shorthand: concatenated CV results across ALL configs
                       # expected columns: ['config','cluster','alpha','l1_wt','score','rank','selected',...]

# ---------------- Winner frequency by hyper-params ----------------
# Filter to the *winning* combo per (config, cluster): rows where `selected == True`.
# Then count how many clusters each (alpha, l1_wt) won within each config.
wins = (
    cv_all[cv_all['selected']]
      .groupby(['config', 'alpha', 'l1_wt'])   # tally wins per config and hyper-param combo
      .size()                                  # number of clusters where this combo was selected
      .rename('wins')                          # name the count column
      .reset_index()                           # turn groupby index back into columns
      .sort_values(['config', 'wins'],        # show most frequent winners first within each config
                   ascending=[True, False])
)
print(wins.head(12))

In [ ]:
# ---------------- “Gap to 2nd best” per cluster ----------------
# This measures how decisively the winning combo beat the runner-up for each (config, cluster).
# Larger gap => more confidence that the chosen hyper-params are truly better for that cluster.

def gap_to_second(g):
    """
    Given a group g containing all CV rows for a single (config, cluster),
    compute the difference between the best and the second-best CV score.
    Return NaN if there aren't at least two candidates.
    """
    s = g.sort_values('score', ascending=False)['score'].to_numpy()
    return np.nan if len(s) < 2 else (s[0] - s[1])

# Apply the gap function per (config, cluster) and summarize by config.
gap = (
    cv_all.groupby(['config', 'cluster'])
          .apply(gap_to_second)                # gap per cluster within each config
          .rename('cv_gap_next')               # name the metric
          .reset_index()
)

# Summary stats (count/mean/std/min/quantiles/max) of gap sizes per config.
gap_summary = gap.groupby('config')['cv_gap_next'].describe()
print(gap_summary)

In [ ]:
# Spot convergence problems quickly
fails = cv_all.query("fit_attempted == True and (fit_ok == False)")
if not fails.empty:
    print("Combos that failed to fit:")
    display(fails[['config','cluster','alpha','l1_wt','error']].head(20))

In [ ]:
best_cfg = summary_df.iloc[0]['config']          # already sorted by your criteria
winners, dist = compare_glm_configs.show_hyperparams_for_config(results, best_cfg)
print('Chosen config:', best_cfg)
print('Per-cluster α/l1_wt:\n', winners)
print('Winning-combo counts:\n', dist)

In [ ]:
# or choose one config
winners, dist = compare_glm_configs.show_hyperparams_for_config(results, 'ElasticNet')
print(winners)   # α & l1_wt per cluster
print(dist)      # how often each combo won

In [ ]:
# CV-table driven reporting across all configs

# winners across all configs (needs the concatenated cv_tables_df returned by sweep_and_compare)
winners_all = (cv_tables_df[cv_tables_df['selected']]
               .groupby(['config','alpha','l1_wt'])
               .size().rename('n_clusters')
               .reset_index()
               .sort_values(['config','n_clusters'], ascending=[True, False]))

# “confidence” in choice per cluster (gap to 2nd best)
def gap_to_second(g):
    s = g.sort_values('score', ascending=False)['score'].to_numpy()
    return np.nan if len(s) < 2 else s[0] - s[1]

cv_gaps = (cv_tables_df.groupby(['config','cluster'])
           .apply(gap_to_second).rename('cv_gap_next').reset_index())
cv_gaps

In [ ]:
# best for cluster
# grab that config’s result dict
best_config = summary_df.iloc[0]['config']
print('Best config:', best_config)

res_best = next(r for r in results if r['config'] == best_config)

# 1) From metrics_df (one row per cluster)
if {'cluster','alpha','l1_wt'}.issubset(res_best['metrics_df'].columns):
    winners = res_best['metrics_df'][['cluster','alpha','l1_wt']].sort_values('cluster')
else:
    # 2) Fallback: from CV grid (selected == True)
    winners = (res_best['cv_tables_df']
               .query('selected')
               [['cluster','alpha','l1_wt','score']]
               .sort_values(['cluster']))
print(winners)

In [ ]:
# Best hyper-params across all configs (using the concatenated CV grid)
# one row per cluster where the winning combo was chosen under each config
chosen = (cv_tables_df.query('selected')
          [['config','cluster','alpha','l1_wt','score']]
          .sort_values(['config','cluster']))
print(chosen.head())

# Appendix

## Iterate through sessions

In [ ]:
# import os, sys, json, hashlib
# from pathlib import Path

# def list_sessions(monkey_dir: Path):
#     return sorted([p.name for p in monkey_dir.iterdir() if p.is_dir() and p.name.startswith('data_')])

# monkey_dir = Path('all_monkey_data/raw_monkey_data/monkey_Bruno')
# all_sessions = list_sessions(monkey_dir)

# for session in all_sessions:
#     print(session)
#     try:
#         raw_data_folder_path = f"all_monkey_data/raw_monkey_data/monkey_Bruno/{session}"
#         cmp_decode.summarize_and_plot_decoding(raw_data_folder_path)   
#     except Exception as e:
#         print(f"Error in {session}: {e}")
#         continue

## Debug ff dataframe

In [ ]:
pn.make_or_retrieve_ff_dataframe()

In [ ]:
pn.ff_dataframe

In [ ]:
pn.ff_dataframe.shape

In [ ]:
h5_file_pathway = os.path.join(os.path.join(
    pn.processed_data_folder_path, 'ff_dataframe.h5'))

h5_file_pathway = 'all_monkey_data/processed_data/monkey_Schro/data_0413/ff_dataframe.h5'

ff_dataframe = pd.read_hdf(h5_file_pathway, 'ff_dataframe')
print("Retrieved ff_dataframe from", h5_file_pathway)
ff_dataframe

## use concat_new_seg_info

In [ ]:
new_seg_info['new_segment'] = np.arange(len(new_seg_info))

In [ ]:
concat_seg_data = pn_utils.concat_new_seg_info(
    pn.monkey_information, new_seg_info, bin_width=0.04)

concat_seg_data['time_since_start_time'] = concat_seg_data['time'] - concat_seg_data['new_seg_start_time']
concat_seg_data['dt'] = np.minimum(concat_seg_data['time_since_start_time'], concat_seg_data['dt'])